In [12]:
%pip install yfinance
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/50/ce/a6bc93f7a44dd1fd23698698e369e141f4f24e7098d0a5937808afee3f5e/matplotlib-3.8.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/8e/ae/a6353db548bff1a592b85ae6bb80275f0a51dc25a0410d059e5b33183e36/contourpy-1.2.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/28/27/42f5cac9f5ee15ab65b3fd939e37a115a0a5e367fcabde5b901599daa85a/fonttools-4.49.0-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.3 kB ? eta -:-


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from matplotlib import pyplot as plt

In [14]:
import numpy as np

In [ ]:
def corr_finder(a, b, t):
    df = yf.download([a,b], start=date.today() - timedelta(5000), end=date.today() + timedelta(1), interval='1d')
    df = df['Adj Close'].resample('D').last().dropna()
    correlation = df[a].pct_change().corr(df[b].pct_change())
    return correlation

variables = ['^GSPC', '^TNX', 'USO', 'JPY=X']
correlations = {}

for a in variables:
    for b in [x for x in variables if x != a]:
        correlations[f'{a}:{b}'] = corr_finder(a, b, 100)

df = pd.DataFrame.from_dict(correlations, orient='index', columns=['correlation'])
df.sort_values('correlation', ascending=False)

In [167]:
#use to to look at the data for 1 ticker
ticker = yf.Ticker('JPY=X')
df = ticker.history(period = '6mo', interval='1d')
df.head()

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-24 00:00:00+01:00,144.673004,145.947006,144.621002,144.673004,0,0.0,0.0
2023-08-25 00:00:00+01:00,146.067001,146.604996,145.733994,146.067001,0,0.0,0.0
2023-08-28 00:00:00+01:00,146.531006,146.716003,146.278000,146.531006,0,0.0,0.0
2023-08-29 00:00:00+01:00,146.464005,147.343994,145.852005,146.464005,0,0.0,0.0
2023-08-30 00:00:00+01:00,145.988007,146.522995,145.557999,145.988007,0,0.0,0.0


In [16]:
# function to format in a useable df, only keeping the date part of datetime to avoid time-caused errors
# also we found that from yfinanve the 'Date' column was actually an index, so converted it to a column
def get_stock_data_to_df(ticker):
    ticker = yf.Ticker(str(ticker))
    df = ticker.history(period = '6mo', interval='1d')
    df.reset_index(inplace=True)
    df["timestamp_processed"] = [e.strftime('%Y-%m-%d') for e in df["Date"]]
    return df

In [17]:
# making the data stionary to fit the assumption. logging the price for the column given and taking the diffence
# created a new column for that called 'stationary'
def stationaryize(df : pd.DataFrame, time) -> pd.DataFrame:
    df['stationary'] = (np.log(df[time])).diff()
    df = df.dropna(how = 'any', axis = 0, )
    return df

In [18]:

def get_clean_data(ticker, time) -> pd.DataFrame:
    return stationaryize(get_stock_data_to_df(ticker.upper()), time)

In [19]:
# tester
aapl = get_clean_data('aapl', "Close")

c:\2.19 Pitch\yearning\.venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [20]:
# main function here which takes in 2 tickers and the open/close/high/low column, and merges the stationary columnms for each on the timestamp column

def merge_df(ticker1, ticker2, open_or_close):
    df1 = get_clean_data(ticker1, open_or_close)
    # print("df of  " + ticker1 + " " + str(np.shape(df1)))
    # print("df of " + ticker1)
    # print(df1.head())
    df2 = get_clean_data(ticker2, open_or_close)
    # print("df of  " + ticker2 + " " + str(np.shape(df2)))
    # print("df of " + ticker2)
    # print(df2.head())
    # return pd.merge(df1[['stationary']], df2[['stationary']], left_index=True, right_index=True, how='right').rename(columns={'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')
    return pd.merge(df1[['timestamp_processed', 'stationary']], df2[['timestamp_processed', 'stationary']], on='timestamp_processed', how='right').rename(columns = {'stationary_x': ticker1, 'stationary_y': ticker2}).dropna(how='any')


In [160]:
merge_df('^TNX', '^GSPC', 'Close')

C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


C:\Users\ashkm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,timestamp_processed,^TNX,^GSPC
0,2023-08-25,0.000944,0.006696
1,2023-08-28,-0.006390,0.006245
2,2023-08-29,-0.021599,0.014404
3,2023-08-30,-0.000971,0.003826
4,2023-08-31,-0.006089,-0.001598
...,...,...,...
120,2024-02-16,0.012888,-0.004815
121,2024-02-20,-0.004667,-0.006023
122,2024-02-21,0.011628,0.001263
123,2024-02-22,0.000462,0.020903


In [ ]:
%pip install statsmodels

In [25]:
from statsmodels.tsa.stattools import grangercausalitytests

In [26]:
# function for the granger test. takes in the tickers, time(open/close/high/low), and lag, and calls merge_df function. returns the test


def granger_causality(ticker1, ticker2, time, lag):
    df = merge_df(ticker1, ticker2, time)
    # if len(df[ticker1]) != len(df[ticker2]):
    # minrows = min(len(df[ticker1]), len(df[ticker2]))
    # df = df.iloc[:,:minrows]
    #print(np.shape(df))
    return grangercausalitytests(df[[ticker1, ticker2]], lag)


In [ ]:
granger_causality('TSLA', 'AAPL', 'Open', 4)

In [ ]:
granger_causality('^TNX', '^GSPC', 'Close', 4)